# Alert Stream and Visits 

Query the number of diaSources reported in `lsst.prompt.prod.numDiaSourcesGood` and merge with visit information.

In [ ]:
import os
from astropy.time import Time, TimeDelta
from IPython.display import display, HTML
from rubin_nights import connections
from rubin_nights.influx_query import InfluxQueryClient
import rubin_nights.dayobs_utils as rn_dayobs

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt

import rubin_nights.plot_utils as rn_plots
colors = rn_plots.PlotStyles.band_colors
markers = rn_plots.PlotStyles.band_symbols

import logging
logging.getLogger('rubin_nights').setLevel(logging.INFO)

In [ ]:
# Connect using your tokenfile and appropriate site 
tokenfile = os.path.join(os.path.expanduser("~"), ".lsst/usdf_rsp")
site = "usdf"
endpoints = connections.get_clients(tokenfile, site)

In [ ]:
day_obs = 20250920

# Find start and end of the night
tstart, tend = rn_dayobs.day_obs_sunset_sunrise(day_obs, sun_alt=-12)

#tstart = Time("2025-09-20T12:00:00")
#tend = Time.now()

sasquatch = InfluxQueryClient("usdfdev", db_name="lsst.prompt")
tt = sasquatch.select_time_series("lsst.prompt.prod.numDiaSourcesGood", 
                                  ["visit", "band", "day_obs", "detector", "dataset_tag", "numAllDiaSources", "numGoodDiaSources", "run"], 
                                  tstart, tend)
if len(tt) > 0:
    alertsum = tt.groupby("visit").agg({'numAllDiaSources': 'sum', 'numGoodDiaSources': ('sum', 'median'), 'detector': 'count'})
    alertsum.index = alertsum.index.astype(int)
    cols = [f"{c[0]}_{c[1]}" for c in alertsum.columns]
    alertsum = alertsum.droplevel(level=0, axis=1)
    alertsum.columns = cols
    alertsum.rename({"detector_count": "nDiaDetectors_count"}, axis=1, inplace=True)
else:
    print("no alerts")

In [ ]:
fields = ['visit', 'exposure', 'detector', 'day_obs', 'ra', 'dec', 'band', 'nPsfStar', 'psfArea', 'psfSigma', 'skyBg', 'skyNoise', 'zeroPoint']
#tt2 = sasquatch.select_time_series("lsst.prompt.prod.initialPviSummaryMetrics", fields, tstart, tend)

In [ ]:
visits = endpoints['consdb'].get_visits("lsstcam", tstart, tend, visit_constraint="science_program = 'BLOCK-365'")

In [ ]:
print("number of block-365 visits:", len(visits))
print("number of visits with alerts:", len(alertsum))

In [ ]:
if len(visits) > 0 and len(alertsum) > 0:
    vv = pd.merge(visits, alertsum, left_on='visit_id', right_index=True, how='outer')
    cols = ['visit_id', 'observation_reason', 'obs_start', 'band', 's_ra', 's_dec', 'sky_rotation', 'clouds', 
        'fwhm_eff', 'cat_m5', 'numGoodDiaSources_sum', 'numGoodDiaSources_median', 'nDiaDetectors_count']
    #display(vv[cols])

In [ ]:
if len(vv) > 0:
    #vv = vv.query("not observation_reason.str.contains('ddf')")
    fig, ax = plt.subplots(1, 2, figsize=(16, 8))
    for tt in ['ddf', 'pair', 'other']:
        if tt == 'other':
            qq = vv.query("~observation_reason.str.contains('pair') and ~observation_reason.str.contains('ddf')")
        else:
            qq = vv.query("observation_reason.str.contains(@tt)")
        if tt == 'ddf':
            marker='^'
        else:
            marker = 'o'
        for b in qq.band.unique():
            dq = qq.query("band == @b")
            for d in dq.day_obs.unique():
                q = dq.query("day_obs == @d")
                if len(q) > 0:
                    if not np.all(np.isnan(q.cat_m5)) and not np.all(np.isnan(q.numGoodDiaSources_sum)):
                        ax[0].scatter(q.cat_m5, q.numGoodDiaSources_sum, s=q.nDiaDetectors_count/189*18 + 5, #color=colors[b], 
                                      marker=marker, linestyle='', label=f"{b} {tt} {d}")
    #ax[0].legend()
    ax[0].set_xlabel("estimated limiting magnitude", fontsize='large')
    ax[0].set_ylabel("nGoodDiaSources per visit", fontsize='large')
    ax[0].set_title(f"Dayobs {day_obs}", fontsize='large')

    for tt in ['ddf', 'pair', 'other']:
        if tt == 'other':
            qq = vv.query("~observation_reason.str.contains('pair') and ~observation_reason.str.contains('ddf')")
        else:
            qq = vv.query("observation_reason.str.contains(@tt)")
        if tt == 'ddf':
            marker='^'
        else:
            marker = 'o'
        for b in qq.band.unique():
            dq = qq.query("band == @b")
            for d in dq.day_obs.unique():
                q = dq.query("day_obs == @d")
                if len(q) > 0:
                    if not np.all(np.isnan(q.cat_m5)) and not np.all(np.isnan(q.numGoodDiaSources_sum)):
                        ax[1].scatter(q.nDiaDetectors_count, q.numGoodDiaSources_sum, s=(q.cat_m5 - q.cat_m5.min())*5 + 9, #color=colors[b], 
                                      marker=marker, linestyle='', label=f"{b} {tt} {d}")
    ax[1].legend(loc=(1.01, 0.1))
    ax[1].set_xlabel("nDiaDetectors per visit")
    ax[1].set_ylabel("nGoodDiaSources per visit", fontsize='large')
    ax[1].set_title(f"Dayobs {day_obs}", fontsize='large')

In [ ]:
fig, ax = plt.subplots(6, 1, figsize=(8, 12), sharex=True)

fig.subplots_adjust(hspace=0.1)
i = 0
for b in vv.band.unique():
    q = vv.query("band == @b")
    ax[i].plot(q.obs_start_mjd, q.fwhm_eff, linestyle='', marker=markers[b], color=colors[b], label=b)
ax[i].plot(vv.obs_start_mjd, vv.fwhm_eff, '-', color='gray', alpha=0.3, zorder=0)
ax[i].legend(loc=(1.01, 0.5))
ax[i].set_ylabel("fwhm (arcsecond)")
ax[i].grid(alpha=0.3)
i += 1
for b in vv.band.unique():
    q = vv.query("band == @b")
    ax[i].plot(q.obs_start_mjd, q.clouds, linestyle='', marker=markers[b], color=colors[b], label=b)
ax[i].plot(vv.obs_start_mjd, vv.clouds, '-', color='gray', alpha=0.3, zorder=0)
ax[i].legend(loc=(1.01, 0.5))
ax[i].set_ylabel("cloud extinction")
ax[i].grid(alpha=0.3)
i += 1
for b in vv.band.unique():
    q = vv.query("band == @b")
    ax[i].plot(q.obs_start_mjd, q.cat_m5, linestyle='', marker=markers[b], color=colors[b], label=b)
ax[i].plot(vv.obs_start_mjd, vv.cat_m5, '-', color='gray', alpha=0.3, zorder=0)
ax[i].legend(loc=(1.01, 0.5))
ax[i].set_ylabel("estimated m5")
ax[i].grid(alpha=0.3)
i += 1
for b in vv.band.unique():
    q = vv.query("band == @b")
    ax[i].plot(q.obs_start_mjd, q.numGoodDiaSources_median*189, linestyle='', marker='o', color='none', markeredgecolor=colors[b])
    ax[i].plot(q.obs_start_mjd, q.numGoodDiaSources_sum, linestyle='', marker=markers[b], color=colors[b], label=b)
ax[i].plot(vv.obs_start_mjd, vv.numGoodDiaSources_sum, '-', color='gray', alpha=0.3, zorder=0)
ax[i].legend(loc=(1.01, 0.5))
ax[i].set_ylabel("Number DiaSources")
ax[i].set_ylim(-10, vv.numGoodDiaSources_sum.max() * 1.2)
ax[i].grid(alpha=0.3)
i += 1
for b in vv.band.unique():
    q = vv.query("band == @b")
    ax[i].plot(q.obs_start_mjd, q.nDiaDetectors_count, linestyle='', marker=markers[b], color=colors[b], label=b)
ax[i].plot(vv.obs_start_mjd, vv.nDiaDetectors_count, '-', color='gray', alpha=0.3, zorder=0)
ax[i].legend(loc=(1.01, 0.5))
ax[i].set_ylabel("NDetectors")
ax[i].set_ylim(0, 189)
ax[i].grid(alpha=0.3)
i += 1
for b in vv.band.unique():
    q = vv.query("band == @b")
    ax[i].plot(q.obs_start_mjd, q.eclip_lat, linestyle='', marker=markers[b], color=colors[b], label=b)
ax[i].plot(vv.obs_start_mjd, vv.eclip_lat, '-', color='gray', alpha=0.3, zorder=0)
ax[i].legend(loc=(1.01, 0.5))
ax[i].set_ylabel("Ecliptic Latitude")
ax[i].grid(alpha=0.3)
_ = fig.suptitle(f"DayObs {day_obs}", y=0.92)